# Verificação do erro:

Importação das bibliotecas necessárias

In [1]:
import json
import re
import numpy as np
import pandas as pd
import open3d as o3d
import modulos

## Leitura do arquivo `results.json`.

Os dados são armazenados como dicionários.

In [2]:
RESULTS_PATH: str = "../Exec 01/results.json"

with open(RESULTS_PATH, "r", encoding="UTF-8") as f:
    results: dict = json.load(f)

## Transformação dos dados de um dicionário para um dataframe do Pandas:

Expressão regular para coletar as nuvens de pontos da chave do dicionário:

In [3]:
regex = re.compile(r"Source: (.*?); Target: (.*)")

## Processando os dados de `results`.

In [4]:
HEADER_RESULTS: list = [
    "Dataset",
    "Scene",
    "Source",
    "Target",
    "Size of Source",
    "Size of Target",
    "Size of Pair",
    "Voxel Size",
    "Run",
    "Method",
    "Alignment Type",
    "TRE",
    "RRE",
]

data_results: list[list] = []

Laço para coletar os dados de `results` do dicionário:

In [5]:
for pair, voxels in results.items():
    match = regex.match(pair)
    if match:
        source_ply_path, target_ply_path = match.groups()

        source_parts = source_ply_path.split("/")
        target_parts = target_ply_path.split("/")

        if len(source_parts) == 5 and len(target_parts) == 5:
            _dataset: str = source_parts[2]
            _scene: str = source_parts[3]

            _source: str = source_parts[4]
            _target: str = target_parts[4]
        else:
            _dataset: str = ""
            _scene: str = ""

            _source: str = ""
            _target: str = ""
            print(
                f"O caminho {source_ply_path} ou {target_ply_path} não está no formato correto."
            )
    else:
        print(f"Não foi possível capturar os caminhos: {pair}")
        continue

    _length_of_source: int = voxels["length of source"]
    _length_of_target: int = voxels["length of target"]
    _length_of_pair: int = _length_of_source + _length_of_target

    for voxel_key, voxel_value in voxels.items():
        if voxel_key.startswith("voxel:"):
            _voxel_size: float = float(voxel_key.replace("voxel: ", ""))
            for run_key, run_value in voxel_value.items():
                _run = int(run_key.replace("run: ", ""))
                for method, method_value in run_value.items():
                    _method: str = method
                    _type: str = "Fine" if "ICP" in _method else "Coarse"
                    _tre: float = float(method_value["tre"])
                    _rre: float = float(method_value["rre"])
                    row = [
                        _dataset,
                        _scene,
                        _source,
                        _target,
                        _length_of_source,
                        _length_of_target,
                        _length_of_pair,
                        _voxel_size,
                        _run,
                        _method,
                        _type,
                        _tre,
                        _rre,
                    ]
                    data_results.append(row)

In [6]:
df_results: pd.DataFrame = pd.DataFrame(data_results, columns=HEADER_RESULTS)

In [7]:
df_results.head()

,Dataset,Scene,Source,Target,Size of Source,Size of Target,Size of Pair,Voxel Size,Run,Method,Alignment Type,TRE,RRE
0,3DMatch,7-scenes-office,cloud_bin_3.ply,cloud_bin_0.ply,313885,484445,798330,0.05,0,RANSAC,Coarse,0.067798,0.057401
1,3DMatch,7-scenes-office,cloud_bin_3.ply,cloud_bin_0.ply,313885,484445,798330,0.05,0,Fast Global Registration,Coarse,0.056938,0.024647
2,3DMatch,7-scenes-office,cloud_bin_3.ply,cloud_bin_0.ply,313885,484445,798330,0.05,0,RANSAC + ICP_Point,Fine,0.029992,0.029250
3,3DMatch,7-scenes-office,cloud_bin_3.ply,cloud_bin_0.ply,313885,484445,798330,0.05,0,RANSAC + ICP_Plane,Fine,0.026330,0.024780
4,3DMatch,7-scenes-office,cloud_bin_3.ply,cloud_bin_0.ply,313885,484445,798330,0.05,0,Fast Global Registration + ICP_Point,Fine,0.013041,0.012789


In [8]:
df_results.describe()

,Size of Source,Size of Target,Size of Pair,Voxel Size,Run,TRE,RRE
count,103644.000000,103644.000000,1.036440e+05,1.036440e+05,103644.000000,103644.000000,93801.000000
mean,280500.130601,253840.762418,5.343409e+05,5.000000e-02,1.500000,0.529735,0.314721
std,149299.675448,131724.380447,2.659304e+05,6.938927e-18,1.118039,1.181013,0.703518
min,64362.000000,64362.000000,1.353910e+05,5.000000e-02,0.000000,0.000282,0.000033
25%,121865.000000,121220.000000,2.423670e+05,5.000000e-02,0.750000,0.015223,0.012017
50%,270699.000000,250104.000000,5.371830e+05,5.000000e-02,1.500000,0.038851,0.024801
75%,389813.000000,333581.000000,7.254680e+05,5.000000e-02,2.250000,0.263479,0.117956
max,760352.000000,729922.000000,1.413787e+06,5.000000e-02,3.000000,24.348717,3.141204


In [9]:
# df_results.query("TRE > 6")
df_results.query("TRE > 0.263479")

,Dataset,Scene,Source,Target,Size of Source,Size of Target,Size of Pair,Voxel Size,Run,Method,Alignment Type,TRE,RRE
100,3DMatch,7-scenes-office,cloud_bin_19.ply,cloud_bin_0.ply,476355,484445,960800,0.05,3,Fast Global Registration,Coarse,0.269781,0.099867
109,3DMatch,7-scenes-office,cloud_bin_47.ply,cloud_bin_0.ply,333581,484445,818026,0.05,0,Fast Global Registration,Coarse,0.554925,0.345399
112,3DMatch,7-scenes-office,cloud_bin_47.ply,cloud_bin_0.ply,333581,484445,818026,0.05,0,Fast Global Registration + ICP_Point,Fine,0.540690,0.332923
113,3DMatch,7-scenes-office,cloud_bin_47.ply,cloud_bin_0.ply,333581,484445,818026,0.05,0,Fast Global Registration + ICP_Plane,Fine,0.486531,0.339908
118,3DMatch,7-scenes-office,cloud_bin_47.ply,cloud_bin_0.ply,333581,484445,818026,0.05,1,Fast Global Registration,Coarse,0.395948,0.265634
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103630,ETH,wood_autmn,Hokuyo_31.ply,Hokuyo_30.ply,108373,106402,214775,0.05,2,Fast Global Registration + ICP_Point,Fine,0.385807,0.036055
103631,ETH,wood_autmn,Hokuyo_31.ply,Hokuyo_30.ply,108373,106402,214775,0.05,2,Fast Global Registration + ICP_Plane,Fine,0.378189,0.038069
103636,ETH,wood_autmn,Hokuyo_31.ply,Hokuyo_30.ply,108373,106402,214775,0.05,3,Fast Global Registration,Coarse,0.500485,0.026330
103639,ETH,wood_autmn,Hokuyo_31.ply,Hokuyo_30.ply,108373,106402,214775,0.05,3,Fast Global Registration + ICP_Point,Fine,0.491766,0.025472


Coleta de todos as matrizes `gt`.

In [10]:
matrizes_gt: dict = {}

for target_ply_path, source_ply_path, t_gt in modulos.get_datasets('../data'):
    key: str = f'Source: {source_ply_path}; Target: {target_ply_path}'.replace("../", "./")
    matrizes_gt[key] = np.array(t_gt)

print(f"Total de matrizes: {len(matrizes_gt)}")
# print(f"Padrão da chave: {key}:")
# print(f"{matrizes_gt[key]}")

Total de matrizes: 2879


Impressão da nuvem de pontos com *source* `cloud_bin_182.ply` e *target* `cloud_bin_69.ply`. 

In [11]:
# TESTE dataset   scene             source              target              voxel   run     method
# 0     3DMatch   7-scenes-office   cloud_bin_3.ply     cloud_bin_0.ply     0.05    0       RANSAC
# TRE:  0.067798  RRE:  0.057401

# 1     3DMatch   7-scenes-office   cloud_bin_182.ply   cloud_bin_69.ply    0.05    0       Robust Global Registration
# TRE:  6.330748  RRE:  2.992489

TESTE: int = 0

dataset: str = "3DMatch"
scene: str = "7-scenes-office"
source: str = ["cloud_bin_3.ply", "cloud_bin_182.ply"][TESTE]
target: str = ["cloud_bin_0.ply", "cloud_bin_69.ply"][TESTE]
source_ply_path: str = f"./data/{dataset}/{scene}/{source}"
target_ply_path: str = f"./data/{dataset}/{scene}/{target}"
key_img: str = f'Source: {source_ply_path}; Target: {target_ply_path}'

voxel_size: float = 0.05
key_voxel: str = f'voxel: {voxel_size}'

run: int = 0
run_key: str = f'run: {run}'

method: str = ["RANSAC", "Robust Global Registration"][TESTE]

Obtenção das transformadas:

In [12]:
transformation: np.ndarray = np.array(results[key_img][key_voxel][run_key][method]["transformation"])

t_gr: np.ndarray = matrizes_gt[key_img]

Comparação visual entre `transformation` (transformação obtida) e `t_gr` (transformação de referência).

In [13]:
print(f"`transformation:`")
print(f"{transformation}")
print(f"`t_gr:`")
print(f"{t_gr}")
print(f"{type(t_gt)}")
print(modulos.tre(transformation, t_gt))
print(modulos.rre(transformation, t_gt))

`transformation:`
[[ 0.97381332  0.18032279 -0.13846048  0.02027989]
 [-0.16908072  0.98156131  0.0891577  -0.50857414]
 [ 0.15198462 -0.06341196  0.98634659  1.07773609]
 [ 0.          0.          0.          1.        ]]
`t_gr:`
[[ 0.9814734   0.13855508 -0.13224826 -0.00301759]
 [-0.12280803  0.98506367  0.12061048 -0.57056978]
 [ 0.14698858 -0.10213504  0.98383631  1.09224127]
 [ 0.          0.          0.          1.        ]]
<class 'numpy.ndarray'>
1.339074188503531
0.16637275013616623


`[0.981473397043	0.13855507584	-0.132248257845	-0.0030175927441]`

`[-0.122808034028	0.985063671726	0.120610475207	-0.570569782637]`
	
`[0.146988582971	-0.102135042662	0.983836308333	1.0922412657]`
	
`[0.0	0.0	0.0	1.0]`	

In [ ]:
_source_ply_path: str = f"../data/{dataset}/{scene}/{source}"
_target_ply_path: str = f"../data/{dataset}/{scene}/{target}"

_source: o3d.geometry.PointCloud = modulos.load_point_cloud(_source_ply_path)
_target: o3d.geometry.PointCloud = modulos.load_point_cloud(_target_ply_path)

modulos.draw_registration_result(_source, _target, t_gt)

## Criação de um dicionário com os pares ordenado em ordem decrescento pelo erro `TRE`.

In [15]:
dict_tre: dict = {}

df_tre: pd.DataFrame = df_results.sort_values(by="TRE", ascending=False)
df_tre.head()

,Dataset,Scene,Source,Target,Size of Source,Size of Target,Size of Pair,Voxel Size,Run,Method,Alignment Type,TRE,RRE
84816,ETH,gazebo_winter,Hokuyo_19.ply,Hokuyo_0.ply,72497,75852,148349,0.05,0,RANSAC,Coarse,24.348717,2.466273
84819,ETH,gazebo_winter,Hokuyo_19.ply,Hokuyo_0.ply,72497,75852,148349,0.05,0,RANSAC + ICP_Plane,Fine,24.339777,2.462154
84818,ETH,gazebo_winter,Hokuyo_19.ply,Hokuyo_0.ply,72497,75852,148349,0.05,0,RANSAC + ICP_Point,Fine,24.331365,2.462941
84809,ETH,gazebo_winter,Hokuyo_18.ply,Hokuyo_0.ply,73738,75852,149590,0.05,3,RANSAC + ICP_Point,Fine,23.010265,3.042607
84807,ETH,gazebo_winter,Hokuyo_18.ply,Hokuyo_0.ply,73738,75852,149590,0.05,3,RANSAC,Coarse,23.002703,3.040757


In [16]:
for index, row in df_tre.iterrows():
    dataset: str = row["Dataset"]
    scene: str = row["Scene"]
    source: str = row["Source"]
    target: str = row["Target"]
    source_ply_path: str = f"./data/{dataset}/{scene}/{source}"
    target_ply_path: str = f"./data/{dataset}/{scene}/{target}"
    key_img: str = f'Source: {source_ply_path}; Target: {target_ply_path}'
    
    if key_img not in dict_tre:
        dict_tre[key_img] = {}
        dict_tre[key_img]["t_gr"] = matrizes_gt[key_img].tolist()
        dict_tre[key_img]["Aprovado"] = None

In [17]:
# OUTPUT_PATH: str = "../checagem_do_alinhamento/checagem.json"
# with open(OUTPUT_PATH, "w", encoding="UTF-8") as f:
#     json.dump(dict_tre, f, indent=4)

In [18]:
print(f"{len(dict_tre)}")

2879
